# Denison CS181/DA210 Homework 2.d - Step 1

Before you turn this problem in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells** (in the menubar, select Kernel$\rightarrow$Restart And Run All).

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

In [49]:
import os
import os.path
import pandas as pd

datadir = "publicdata"

---

## Part A: Aggregating a `Series`/`DataFrame`

**Q1:** Read CSV file `topnames.csv` in `datadir` into a data frame named `topnames0`, with no index.  Using individual operations on the `count` column, find the mean, the median, and the max count, assigning to `mean_counts`, `median_counts`, and `max_counts`.

In [2]:
filepath = os.path.join(datadir, 'topnames.csv')
topnames0 = pd.read_csv(filepath)
mean_counts = topnames0["count"].mean()
median_counts = topnames0["count"].median()
max_counts = topnames0["count"].max()

In [3]:
# Printing cell
print("Mean:", mean_counts,
      "Median:", median_counts,
      "Max:", max_counts)

Mean: 44985.24460431655 Median: 52556.0 Max: 99689


In [4]:
# Testing Cell
assert abs(mean_counts - 44985.2446) < 0.001
assert median_counts == 52556.0
assert max_counts == 99689

**Q2:** Using the `agg` method on the `Series` of the column vector of counts, perform the same calculation of mean, median, and max in a single step, and assign to `agg_values`.  Note that this invocation may not have an exact correspondent in the book, so you may have to look up documentation of using `agg` on a `Series`.

Note in a comment in the code solution cell the **data type** of the result.

In [5]:
agg_values = topnames0["count"].agg(['mean', 'median', 'max'])

# Data type of result: series
# TODO

# Display the aggregated values
agg_values

mean      44985.244604
median    52556.000000
max       99689.000000
Name: count, dtype: float64

In [6]:
# Testing Cell
assert abs(agg_values.loc["mean"] - 44985.2446) < 0.001
assert agg_values.loc["median"] == 52556.0
assert agg_values.loc["max"] == 99689.0

**Q3:** Create a subset of `topnames0` restricted to `Female` entries between 1960 and 1969, inclusive.  Assign this to `female_subset`.  Then use the `agg` function, in one step, to determine the mean and median count and the number of unique names, assigning to `female_aggvalues`.

In a comment in the code solution cell, indicate the data type of the result.

In [16]:
female_subset = topnames0.loc[topnames0.sex == "Female", :]
female_subset = female_subset.loc[female_subset.year >= 1960, :]
female_subset = female_subset.loc[female_subset.year <= 1969, :]
female_aggvalues = female_subset.agg({'count':['mean', 'median'], 'name':'nunique'})

# Data type of result: dataframe
# TODO

# Display the aggregated values
female_aggvalues

,count,name
mean,51972.7,NaN
median,51955.0,NaN
nunique,NaN,2.0


In [17]:
# Testing Cell
assert female_aggvalues.shape == (3,2)
assert female_aggvalues.loc["mean", "count"] == 51972.7
assert female_aggvalues.loc["median", "count"] == 51955.0
assert female_aggvalues.loc["nunique", "name"] == 2.0

**Q4:** The constraints for selecting the rows from the last problem are based on `sex` and `year`.  We often use these independent variables to set an index for a data set.  Then, when we want to filter rows, our operations that use row label/`index` values for filtering are different.

Start by creating dataframe `topnames` with its index drawn from the columns `year` and `sex`.  Then,
with a goal of the same use of the `agg` function from **Q3**, use `xs` to take a cross section of `topnames` to get the Female entries and then use `loc` to get a data frame, `female_xs_subset`.  Finally, use `agg` on this data frame to, in one step, determine the mean and median count and the number of unique names, assigning to `female_xs_aggvalues`.

In [39]:
topnames = topnames0.set_index(['sex', 'year'])
topnames = topnames.xs(key = "Female", level = "sex")
female_xs_subset = topnames.loc[1960:1969, :]
female_xs_aggvalues = female_xs_subset.agg({"count":["mean", "median"], "name":"nunique"})
# Display the aggregated values
female_xs_aggvalues


,count,name
mean,51972.7,NaN
median,51955.0,NaN
nunique,NaN,2.0


In [40]:
# Testing Cell
assert female_xs_aggvalues.shape == (3,2)
assert female_xs_aggvalues.loc["mean", "count"] == 51972.7
assert female_xs_aggvalues.loc["median", "count"] == 51955.0
assert female_xs_aggvalues.loc["nunique", "name"] == 2.0

---

## Part B: GroupBy

**Q5** Read CSV file `indicators2016.csv` in `datadir` into a data frame named `indicators0`, with no index. Write code to add a new column `popSize` to `indicators0` which takes value 'high' if `pop > 300`, 'low' if `pop < 50`, and 'medium' otherwise. 

In [60]:
filepath = os.path.join(datadir, "indicators2016.csv")
indicators0 = pd.read_csv(filepath)
indicators0["popSize"] = indicators0["pop"].apply(lambda s: "low" if s < 50 else ("high" if s > 300 else "medium"))

# Display part of the DataFrame
indicators0.head()

,code,country,pop,gdp,life,cell,popSize
0,ABW,Aruba,0.10,NaN,75.87,NaN,low
1,AFG,Afghanistan,34.66,19.47,63.67,21.60,low
2,AGO,Angola,28.81,95.34,61.55,13.00,low
3,ALB,Albania,2.88,11.86,78.34,3.37,low
4,AND,Andorra,0.08,2.86,NaN,0.07,low


In [61]:
# Testing Cell
assert indicators0["popSize"].nunique() == 3
assert indicators0["popSize"].value_counts()["low"] == 189
assert indicators0["popSize"].value_counts()["medium"] == 25
assert indicators0["popSize"].value_counts()["high"] == 6
assert indicators0.loc[205, "popSize"] == "high"
assert indicators0.loc[17, "popSize"] == "medium"
assert indicators0.loc[4, "popSize"] == "low"

**Q6:** Building on the question above, use `groupby` to partition `indicators0` by this new column `popSize`, assigning to variable `groupby_pop`.  Note in a comment the data type of `groupby_pop`.

In [62]:
groupby_pop = indicators0.groupby("popSize")

# Data type of groupby_pop: groupby
# TODO

# Print the number of groups
print(len(groupby_pop))

3


In [63]:
# Testing Cell
assert len(groupby_pop) == 3

**Q7:** Building on the question above, aggregate this groupby partitioning, determining the **number of non-missing elements** for each of the columns by partition.  Assign to `partition_counts` and also include a comment giving the data type of the result, and the row labels of the result.

Hint: You can make this general by using a list-comprehension-type expression to create a dictionary.  (This is not necessary, but cool to use.)

In [70]:
partition_counts = groupby_pop.agg("count")

# Display the grouped counts
partition_counts

,code,country,pop,gdp,life,cell
popSize,,,,,,
high,6,6,6,6,6,6
low,189,189,189,164,171,172
medium,25,25,24,24,25,25


In [69]:
# Testing Cell
assert partition_counts.shape == (3,7)
assert partition_counts.loc["low", "country"] == 189
assert partition_counts.loc["medium", "gdp"] == 24
assert partition_counts.loc["high", "cell"] == 6

AssertionError: 

**Q8:** In similar fashion, determine the mean, by partition, of `pop`, `gdp`, and `cell`, and the max of `life`, assigning to `partition_aggvalues`.  Use the `round()` method of DataFrames to round the numeric values to 2 decimal places.

In [76]:
partition_aggvalues = groupby_pop.agg({"pop" : "mean", "gdp" : "mean", "cell" : "mean", "life" : "max"})
partition_aggvalues = partition_aggvalues.round(decimals = 2)

# Display the grouped aggregated values
partition_aggvalues

,pop,gdp,cell,life
popSize,,,,
high,2100.92,18715.31,2040.46,78.69
low,9.44,101.67,10.26,84.23
medium,108.52,1076.00,113.07,83.98


In [77]:
# Testing Cell
assert partition_aggvalues.shape == (3,4)
assert partition_aggvalues.loc["low", "gdp"] == 101.67
assert partition_aggvalues.loc["medium", "life"] == 83.98
assert partition_aggvalues.loc["high", "pop"] == 2100.92

**Q9:** In similar fashion, determine the mean, min, and max of `gdp` and `life`, assigning to `partition_aggvalues2` and again rounding to two decimal places.

In [80]:
partition_aggvalues2 = groupby_pop.agg({"gdp" : ["mean", "min", "max"], "life" : ["mean", "min", "max"]})
partition_aggvalues2 = partition_aggvalues2.round(decimals = 2)
# Display the grouped aggregated values
partition_aggvalues2

gdp                      life              
             mean      min       max   mean    min    max
popSize                                                  
high     18715.31  1440.24  75871.74  72.87  68.56  78.69
low        101.67     0.03   1535.77  72.15  51.84  84.23
medium    1076.00    35.38   4949.27  72.29  53.43  83.98

In [81]:
# Testing Cell
assert partition_aggvalues2.shape == (3,6)
assert partition_aggvalues2.loc["low", ("gdp", "mean")] == 101.67
assert partition_aggvalues2.loc["medium", ("life", "max")] == 83.98
assert partition_aggvalues2.loc["high", ("gdp", "min")] == 1440.24

**Q10:** Explain how the columns of `partition_aggvalues2` differ from  how the columns of this result differ from the results of Q8 and Q9, and **why**.

Q10 columns differ from Q8 in that they display different aggregated values (mean, mix, max of 2 columns vs count of non missing values of each of all columns)
Q10 columns differ from Q9 in that they also display different aggregated values (mean, min, max of 2 columns vs mean or max of 4 columns). They also have different mapping (3 columns under each aggregated column vs 1 column under each aggregated column)

---

---
## Part C

**Q11:** How much time (in minutes/hours) did you spend on this homework assignment?

3.5 hours

**Q12:** Who was your partner for this assignment?  If you worked alone, say so instead.

I worked alone